# pre_processing

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/train.csv")

In [13]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [4]:
def load_train():
    df = pd.read_csv('./data/train.csv', index_col='id', parse_dates=['fecha'])
    df = df[['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
         'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
         'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio']]
    
    whitelist_tipodepropiedad = df['tipodepropiedad'].value_counts().index.to_list()[:9]
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: x if x in whitelist_tipodepropiedad else 'Otro') 
    
    whitelist_ciudad = df['ciudad'].value_counts().index.to_list()[:80]
    df['ciudad'] = df['ciudad'].apply(lambda x: x if x in whitelist_ciudad else 'Otro')
    
    categoricas = ['tipodepropiedad', 'ciudad', 'provincia']
    df[categoricas] = df[categoricas].fillna('unknown')
    df[categoricas] = df[categoricas].astype('category')
    enteras = ['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']
    df[enteras] = df[enteras].astype('int8')
    
    return df

In [3]:
def load_test():
    df = pd.read_csv('./data/test.csv', index_col='id', parse_dates=['fecha'])
    df = df[['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad',
         'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
         'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']]
    
    whitelist_tipodepropiedad = df['tipodepropiedad'].value_counts().index.to_list()[:9]
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: x if x in whitelist_tipodepropiedad else 'Otro') 
    
    whitelist_ciudad = df['ciudad'].value_counts().index.to_list()[:80]
    df['ciudad'] = df['ciudad'].apply(lambda x: x if x in whitelist_ciudad else 'Otro')
    
    whitelist = df['tipodepropiedad'].value_counts().index.to_list()[:9]
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: x if x in whitelist else 'Otro')    
    categoricas = ['tipodepropiedad', 'ciudad', 'provincia']
    df[categoricas] = df[categoricas].fillna('unknown')
    df[categoricas] = df[categoricas].astype('category')
    enteras = ['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']
    df[enteras] = df[enteras].astype('int8')
    
    return df